<a href="https://colab.research.google.com/github/jlc06/FantasyFootball/blob/main/FFBallWebScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Functions

In [ ]:
# This function is used to input new rows into the existing dataframe created using the below prodata function.
def Insert_row_(row_number, df, row_value):
    import pandas as pd
    # Slice the upper half of the dataframe
    ddf = df[0:row_number]

    # Store the result of lower half of the dataframe
    dff = df[row_number:]

    # Insert the row in the upper half dataframe
    #ddf.loc[row_number]=row_value
    ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)

    # Concat the two dataframes
    df_result = pd.concat([ddf, dff])

    # Reassign the index labels
    df_result.index = [*range(df_result.shape[0])]

    # Return the updated dataframe
    return df_result

In [ ]:
# This function scrapes the pro-football-reference fantasy stats, cleans the data, and calculates additional statistics
# pids refers to a static file that assigns a unique id to each player given different iterations of their names on different data sources
def prodata(year,pids):
    # Import dependencies
    import time
    import pandas as pd
    from bs4 import BeautifulSoup
    import requests
    import numpy as np
    #Scrape and create dataframe from website
    url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
    df = pd.read_html(url,header=1)[0]
    df = df[df['FantPos'] != 'FantPos'] # removes unnecessary rows
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    ids = soup.find_all('td',attrs={"data-stat":"player"}) # finds unique player id embedded in webiste to search for their unique player page if necessary
    playerids = []
    hrefs = []
    for id in ids:
      playerid = id.get('data-append-csv')
      playerids.append(playerid)
      for part in id.find_all('a'):
        href = part.get('href')
        hrefs.append(href)
    df['playerid'] = playerids
    df['href'] = hrefs
    column_to_move = df.pop('playerid')
    df.insert(0,'playerid',column_to_move)
    column_to_move = df.pop('href')
    df.insert(0,'href',column_to_move)
    df['Passes'] = df['Att']
    df['Rushes'] = df['Att.1']
    df['Targets'] = df['Tgt']
    df['Passes'] = pd.to_numeric(df['Passes'],errors='coerce')
    df['Rushes'] = pd.to_numeric(df['Rushes'],errors='coerce')
    df['Targets'] = pd.to_numeric(df['Targets'],errors='coerce')
    df['OppsCheck'] = (df['Passes']) + (df['Rushes']) + (df['Targets'])
    df = df[df['OppsCheck'] > 0 ] # removes players that don't have any opportunities
    # Drop unnecessary columns
    df = df.drop(['Rk','VBD','FantPt','PPR','DKPt','FDPt','PosRank','OvRank','Passes','Rushes','Targets','OppsCheck'],axis=1)
    df = df[df['FantPos'].notna()]
    df = df.reset_index(drop=True)
    df['DupFlag'] = 1
    mult_tm_pl = len(df[(df['Tm'] == '2TM') | (df['Tm'] == '3TM') | (df['Tm'] == '4TM')]) # finds players who have played on multiple teamms
    print(f'Estimated time to completion: {float((mult_tm_pl * 12)/60)} minutes') #rough estimate
    # Add rows of players who have played on multiple teams in-season
    added_rows = 0
    for index, row in df.iterrows():
      if row['Tm'] == '2TM' or row['Tm'] == '3TM' or row['Tm'] == '4TM':
        href = row['href']
        playerid = row['playerid']
        Player = row['Player']
        print(Player)
        FantPos = row['FantPos']
        Age = row['Age']
        cur_index = index + added_rows
        num_tms = (int(row['Tm'][0]))
        temp_href = row['href']
        temp_url = f'https://www.pro-football-reference.com/{temp_href}'
        try:
          pldfpass = pd.read_html(temp_url,header=0,attrs={'id':'passing'})[0]
          try:
            pldfrr = pd.read_html(temp_url,header=1,attrs={'id':'rushing_and_receiving'})[0]
            table = 'rushrec'
          except:
            pldfrr = pd.read_html(temp_url,header=1,attrs={'id':'receiving_and_rushing'})[0]
            table = 'recrush'
          pldfpass['Year'] = pldfpass['Year'].replace('\+','',regex=True).astype(object)
          pldfpass['Year'] = pldfpass['Year'].replace('\*','',regex=True).astype(object)
          pldfpass['Year'] = pldfpass['Year'].str.rstrip()
          pldfrr['Year'] = pldfrr['Year'].replace('\+','',regex=True).astype(object)
          pldfrr['Year'] = pldfrr['Year'].replace('\*','',regex=True).astype(object)
          pldfrr['Year'] = pldfrr['Year'].str.rstrip()
          year_index_pass = pldfpass.index[pldfpass['Year'] == str(year)][0]
          year_index_rr = pldfrr.index[pldfrr['Year'] == str(year)][0]
          top_pass = year_index_pass + 1
          top_rr = year_index_rr + 1
          bottom_pass = year_index_pass + num_tms
          bottom_rr = year_index_rr + num_tms
          temp_pass_df = pldfpass.iloc[top_pass:bottom_pass + 1,:]
          temp_rr_df = pldfrr.iloc[top_rr:bottom_rr + 1,:]
          temp_pass_df = temp_pass_df[(temp_pass_df['Year'] == '') | (temp_pass_df['Year'].isna())]
          temp_rr_df = temp_rr_df[(temp_rr_df['Year'] == '') | (temp_rr_df['Year'].isna())]
          temp_pass_df = temp_pass_df.set_index('Tm')
          temp_rr_df = temp_rr_df.set_index('Tm')
          if len(temp_pass_df) >= len(temp_rr_df):
            temp_df = temp_pass_df.join(temp_rr_df,lsuffix='_pass', rsuffix='_rr')
          else:
            temp_df = temp_rr_df.join(temp_pass_df,lsuffix='_rr', rsuffix='_pass')
          temp_df.reset_index(inplace=True)
          temp_df = temp_df.rename(columns = {'index':'Tm'})
          for id in range(0,len(temp_df)):
            href = href
            playerid = playerid
            Player = Player
            Tm = temp_df.iloc[id]['Tm']
            FantPos = FantPos
            Age = Age
            if len(temp_pass_df) >= len(temp_rr_df):
              G = temp_df.iloc[id]['G_pass']
              GS = temp_df.iloc[id]['GS_pass']
            else:
              G = temp_df.iloc[id]['G_rr']
              GS = temp_df.iloc[id]['GS_rr']
            pass_cmp = temp_df.iloc[id]['Cmp']
            pass_att = temp_df.iloc[id]['Att_pass']
            pass_yds = temp_df.iloc[id]['Yds_pass']
            pass_td = temp_df.iloc[id]['TD_pass']
            pass_int = temp_df.iloc[id]['Int']
            if table == 'rushrec':
              rush_att = temp_df.iloc[id]['Att_rr']
              rush_yds = temp_df.iloc[id]['Yds_rr']
              rush_ya = temp_df.iloc[id]['Y/A_rr']
              rush_td = temp_df.iloc[id]['TD_rr']
              rec_tgt = temp_df.iloc[id]['Tgt']
              rec_rec = temp_df.iloc[id]['Rec']
              rec_yds = temp_df.iloc[id]['Yds.1_rr']
              rec_yr = temp_df.iloc[id]['Y/R']
              rec_td = temp_df.iloc[id]['TD.1']
            else:
              rush_att = temp_df.iloc[id]['Att_rr']
              rush_yds = temp_df.iloc[id]['Yds.1_rr']
              rush_ya = temp_df.iloc[id]['Y/A_rr']
              rush_td = temp_df.iloc[id]['TD.1']
              rec_tgt = temp_df.iloc[id]['Tgt']
              rec_rec = temp_df.iloc[id]['Rec']
              rec_yds = temp_df.iloc[id]['Yds_rr']
              rec_yr = temp_df.iloc[id]['Y/R']
              rec_td = temp_df.iloc[id]['TD_rr']
            rush_fmb = temp_df.iloc[id]['Fmb']
            rush_fl = rush_fmb
            total_td = int(pass_td) + int(rush_td) + int(rec_td)
            twopm = 0
            twopp = 0
            dupflag = 0
            lst = [href,playerid,Player,Tm,FantPos,Age,G,GS,pass_cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_ya,rush_td,rec_tgt,rec_rec,rec_yds,rec_yr,rec_td,rush_fmb,rush_fl,total_td,twopm,twopp,dupflag]
            df = Insert_row_(cur_index + 1 + id,df,lst)
          added_rows += len(temp_df)
          time.sleep(12)
        except:
          try:
            pldfrr = pd.read_html(temp_url,header=1,attrs={'id':'rushing_and_receiving'})[0]
            table = 'rushrec'
          except:
            pldfrr = pd.read_html(temp_url,header=1,attrs={'id':'receiving_and_rushing'})[0]
            table = 'recrush'
          pldfrr['Year'] = pldfrr['Year'].replace('\+','',regex=True).astype(object)
          pldfrr['Year'] = pldfrr['Year'].replace('\*','',regex=True).astype(object)
          pldfrr['Year'] = pldfrr['Year'].str.rstrip()
          year_index = pldfrr.index[pldfrr['Year'] == str(year)][0]
          top = year_index + 1
          bottom = year_index + num_tms
          temp_df = pldfrr.iloc[top:bottom + 1,:]
          temp_df = temp_df[(temp_df['Year'] == '') | (temp_df['Year'].isna())]
          temp_df = temp_df.set_index('Tm')
          temp_df.reset_index(inplace=True)
          temp_df = temp_df.rename(columns = {'index':'Tm'})
          for id in range(0,len(temp_df)):
            href = href
            playerid = playerid
            Player = Player
            Tm = temp_df.iloc[id]['Tm']
            FantPos = FantPos
            Age = Age
            G = temp_df.iloc[id]['G']
            GS = temp_df.iloc[id]['GS']
            pass_cmp = 0
            pass_att = 0
            pass_yds = 0
            pass_td = 0
            pass_int = 0
            if table == 'rushrec':
              rush_att = temp_df.iloc[id]['Att']
              rush_yds = temp_df.iloc[id]['Yds']
              rush_ya = temp_df.iloc[id]['Y/A']
              rush_td = temp_df.iloc[id]['TD']
              rec_tgt = temp_df.iloc[id]['Tgt']
              rec_rec = temp_df.iloc[id]['Rec']
              rec_yds = temp_df.iloc[id]['Yds.1']
              rec_yr = temp_df.iloc[id]['Y/R']
              rec_td = temp_df.iloc[id]['TD.1']
            else:
              rush_att = temp_df.iloc[id]['Att']
              rush_yds = temp_df.iloc[id]['Yds.1']
              rush_ya = temp_df.iloc[id]['Y/A']
              rush_td = temp_df.iloc[id]['TD.1']
              rec_tgt = temp_df.iloc[id]['Tgt']
              rec_rec = temp_df.iloc[id]['Rec']
              rec_yds = temp_df.iloc[id]['Yds']
              rec_yr = temp_df.iloc[id]['Y/R']
              rec_td = temp_df.iloc[id]['TD']
            rush_fmb = temp_df.iloc[id]['Fmb']
            rush_fl = rush_fmb
            total_td = int(pass_td) + int(rush_td) + int(rec_td)
            twopm = 0
            twopp = 0
            dupflag = 0
            lst = [href,playerid,Player,Tm,FantPos,Age,G,GS,pass_cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_ya,rush_td,rec_tgt,rec_rec,rec_yds,rec_yr,rec_td,rush_fmb,rush_fl,total_td,twopm,twopp,dupflag]
            df = Insert_row_(cur_index + 1 + id,df,lst)
          added_rows += len(temp_df)
          time.sleep(12)
      else:
        continue
    # Clean up the dataframe
    df['Cmp'] = df['Cmp'].fillna(0)
    df['Att'] = df['Att'].fillna(0)
    df['Yds'] = df['Yds'].fillna(0)
    df['TD'] = df['TD'].fillna(0)
    df['Int'] = df['Int'].fillna(0)
    df['Att.1'] = df['Att.1'].fillna(0)
    df['Yds.1'] = df['Yds.1'].fillna(0)
    df['Y/A'] = df['Y/A'].fillna(0)
    df['Tgt'] = df['Tgt'].fillna(0)
    df['Rec'] = df['Rec'].fillna(0)
    df['Yds.2'] = df['Yds.2'].fillna(0)
    df['Y/R'] = df['Y/R'].fillna(0)
    df['2PM'] = df['2PM'].fillna(0)
    df['2PP'] = df['2PP'].fillna(0)
    df['Fmb'] = df['Fmb'].fillna(0)
    df['FL'] = df['FL'].fillna(0)
    df = df.replace('\+','',regex=True).astype(object)
    df = df.replace('\*','',regex=True).astype(object)
    # Strips extra whitespace from end of player column
    df['Player'] = df['Player'].str.rstrip()
    # Update total tds if null
    for index, row in df.iterrows():
      if(pd.isnull(row['TD.3'])):
        row['TD.3'] = row['TD'] + row['TD.1'] + row['TD.2']
    # Change dtypes of columns to numeric
    columns = df.columns[5:]
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    # Rank players
    df['.5PPR'] = (df['Yds']*.04)+(df['TD']*4)+(df['Int']*-2)+(df['Yds.1']*.1)+(df['TD.1']*6)+(df['Rec']*.5)+(df['Yds.2']*.1)+(df['TD.2']*6)+(df['FL']*-2)+(df['2PM']*2)+(df['2PP']*2)+((df['TD.3']-(df['TD.1']+df['TD.2']))*6)
    df['.5PPR'] = df['.5PPR'].round(decimals = 1)
    df['PPG'] = df['.5PPR'] / df['G']
    df['PPG'] = df['PPG'].round(decimals = 1)
    df['.5PosRank'] = df.query("DupFlag == 1").groupby('FantPos')['.5PPR'].rank(method='first',ascending=False)
    df['.5PosRank'] = df['.5PosRank'].fillna(0)
    df['.5PosRank'] = df['.5PosRank'].astype(int)
    df.loc[df.eval("DupFlag == 1"), '.5PRank'] = df['FantPos'] + df['.5PosRank'].astype(str)
    df['.5PRank'] = df['.5PRank'].fillna('0')
    df['QBPassAttRank'] = df['Att'][(df['DupFlag'] == 1) & (df['FantPos']=='QB')].rank(method='first',ascending=False)
    df['QBPassAttRank'] = df['QBPassAttRank'].fillna(0)
    df['QBPassAttRank'] = df['QBPassAttRank'].astype(int)
    df['RBRushRank'] = df['Att.1'][(df['DupFlag'] == 1) & (df['FantPos']=='RB')].rank(method='first',ascending=False)
    df['RBRushRank'] = df['RBRushRank'].fillna(0)
    df['RBRushRank'] = df['RBRushRank'].astype(int)
    df['WRTETgtRank'] = df['Tgt'][(df['DupFlag'] == 1) & ((df['FantPos']=='WR') | (df['FantPos']=='TE'))].rank(method='first',ascending=False)
    df['WRTETgtRank'] = df['WRTETgtRank'].fillna(0)
    df['WRTETgtRank'] = df['WRTETgtRank'].astype(int)
    df['FlexTgtRank'] = df['Tgt'][(df['DupFlag'] == 1) & ((df['FantPos']=='WR') | (df['FantPos']=='TE') | (df['FantPos']=='RB'))].rank(method='first',ascending=False)
    df['FlexTgtRank'] = df['FlexTgtRank'].fillna(0)
    df['FlexTgtRank'] = df['FlexTgtRank'].astype(int)
    df['TotalOpps'] = (df['Att']) + (df['Att.1']) + (df['Tgt'])
    df['QBOppsRank'] = df['TotalOpps'][(df['DupFlag'] == 1) & (df['FantPos'] == 'QB')].rank(method='first',ascending=False)
    df['QBOppsRank'] = df['QBOppsRank'].fillna(0)
    df['QBOppsRank'] = df['QBOppsRank'].astype(int)
    df['RBOppsRank'] = df['TotalOpps'][(df['DupFlag'] == 1) & (df['FantPos'] == 'RB')].rank(method='first',ascending=False)
    df['RBOppsRank'] = df['RBOppsRank'].fillna(0)
    df['RBOppsRank'] = df['RBOppsRank'].astype(int)
    df['WROppsRank'] = df['TotalOpps'][(df['DupFlag'] == 1) & (df['FantPos'] == 'WR')].rank(method='first',ascending=False)
    df['WROppsRank'] = df['WROppsRank'].fillna(0)
    df['WROppsRank'] = df['WROppsRank'].astype(int)
    df['TEOppsRank'] = df['TotalOpps'][(df['DupFlag'] == 1) & (df['FantPos'] == 'TE')].rank(method='first',ascending=False)
    df['TEOppsRank'] = df['TEOppsRank'].fillna(0)
    df['TEOppsRank'] = df['TEOppsRank'].astype(int)
    df['Pts/Opp'] = df['.5PPR'] / df['TotalOpps']
    df['Pts/Opp'] = df['Pts/Opp'].fillna(0.0)
    df['P/ORank'] = df['Pts/Opp'][(df['DupFlag'] == 1) & (df['TotalOpps'] >= 30)].rank(method='first',ascending=False)
    df['P/ORank'] = df['P/ORank'].fillna(0)
    df['P/ORank'] = df['P/ORank'].astype(int)
    df = df.replace(-np.Inf,0)
    team_rushes = {}
    rb_rushes = {}
    team_passes = {}
    team_targets = {}
    wrte_targets ={}
    skill_offense = {}
    qb_offense = {}
    keys = []
    for index, row in df.iterrows():
      if row['Tm'] in keys:
        continue
      else:
        keys.append(row['Tm'])
    for i in keys:
      team_rushes[i] = 0
      rb_rushes[i] = 0
      team_passes[i] = 0
      team_targets[i] = 0
      wrte_targets[i] = 0
      skill_offense[i] = 0
      qb_offense[i] = 0
    for index, row in df.iterrows():
      team_rushes[row['Tm']] += row['Att.1']
      team_passes[row['Tm']] += row['Att']
      team_targets[row['Tm']] += row['Tgt']
      skill_offense[row['Tm']] += (row['Att.1'] + row['Tgt'])
      qb_offense[row['Tm']] += (row['Att'] + row['Att.1'])
      if row['FantPos'] == 'RB':
        rb_rushes[row['Tm']] += row['Att.1']
      elif row['FantPos'] == 'WR' or row['FantPos'] == 'TE':
        wrte_targets[row['Tm']] += row['Tgt']
      else:
        continue
    #print(team_rushes)
    #print(team_passes)
    #print(team_targets)
    #print(rb_rushes)
    #print(wrte_targets)
    totrushshare = []
    rbrushshare = []
    tottgtshare = []
    wrtetgtshare = []
    rbtewroshare = []
    qboshare = []
    for index, row in df.iterrows():
      tm = row['Tm']
      if tm == '2TM' or tm == '3TM' or tm == '4TM':
        totrushshare.append(0)
      else:
        rushatt = row['Att.1']
        share = rushatt / team_rushes[tm]
        totrushshare.append(share)
    for index, row in df.iterrows():
      tm = row['Tm']
      if tm == '2TM' or tm == '3TM' or tm == '4TM':
        tottgtshare.append(0)
      else:
        tgt = row['Tgt']
        share = tgt / team_targets[tm]
        tottgtshare.append(share)
    for index, row in df.iterrows():
      tm = row['Tm']
      if tm == '2TM' or tm == '3TM' or tm == '4TM':
        rbrushshare.append(0)
      elif row['FantPos'] != 'RB':
        rbrushshare.append(0)
      else:
        rushatt = row['Att.1']
        share = rushatt / rb_rushes[tm]
        rbrushshare.append(share)
    for index, row in df.iterrows():
      tm = row['Tm']
      if tm == '2TM' or tm == '3TM' or tm == '4TM':
        wrtetgtshare.append(0)
      elif row['FantPos'] == 'WR' or row['FantPos'] == 'TE':
        tgt = row['Tgt']
        share = tgt / wrte_targets[tm]
        wrtetgtshare.append(share)
      else:
        wrtetgtshare.append(0)
    for index, row in df.iterrows():
      tm = row['Tm']
      if tm == '2TM' or tm == '3TM' or tm == '4TM':
        rbtewroshare.append(0)
      elif row['FantPos'] == 'RB' or row['FantPos'] == 'WR' or row['FantPos'] == 'TE':
        off = row['Att.1'] + row['Tgt']
        share = off / skill_offense[tm]
        rbtewroshare.append(share)
      else:
        rbtewroshare.append(0)
    for index, row in df.iterrows():
      tm = row['Tm']
      if tm == '2TM' or tm == '3TM' or tm == '4TM':
        qboshare.append(0)
      elif row['FantPos'] == 'QB':
        off = row['Att'] + row['Att.1']
        share = off / qb_offense[tm]
        qboshare.append(share)
      else:
        qboshare.append(0)
    df['TotRushShare'] = totrushshare
    df['RBRushShare'] = rbrushshare
    df['TotTgtShare'] = tottgtshare
    df['WR/TETgtShare'] = wrtetgtshare
    df['SkillOffShare'] = rbtewroshare
    df['QBOffShare'] = qboshare
    df['NonTDPts'] = (df['Yds']*.04)+(df['Int']*-2)+(df['Yds.1']*.1)+(df['Rec']*.5)+(df['Yds.2']*.1)+(df['FL']*-2)+(df['2PM']*2)+(df['2PP']*2)
    df['NonTDPts'] = df['NonTDPts'].round(decimals = 1)
    df['NonTDPosRank'] = df.query("DupFlag == 1").groupby('FantPos')['NonTDPts'].rank(method='first',ascending=False)
    df['NonTDPosRank'] = df['NonTDPosRank'].fillna(0)
    df['NonTDPosRank'] = df['NonTDPosRank'].astype(int)
    df.loc[df.eval("DupFlag == 1"), 'NonTDPRank'] = df['FantPos'] + df['NonTDPosRank'].astype(str)
    df['NonTDPRank'] = df['NonTDPRank'].fillna('0')
    # Create year and PPG columns
    df['Year'] = year
    cols = []
    for column in df.columns:
      if column == 'FantPos':
        cols.append('Pos')
      elif column == 'Yds':
        cols.append('PassYds')
      elif column == 'Yds.1':
        cols.append('RushYds')
      elif column == 'Yds.2':
        cols.append('RecYds')
      elif column == 'Att':
        cols.append('PassAtt')
      elif column == 'Att.1':
        cols.append('RushAtt')
      elif column == 'TD':
        cols.append('PassTD')
      elif column == 'TD.1':
        cols.append('RushTD')
      elif column == 'TD.2':
        cols.append('RecTD')
      elif column == 'TD.3':
        cols.append('TotTD')
      elif column == 'Y/A':
        cols.append('RY/A')
      else:
        cols.append(column)
    df.columns = cols
    ids = []
    for index, row in df.iterrows():
      player = row['Player']
      for index, row in pids.iterrows():
        if row['1'] == player:
          ids.append(row['PlayerId'])
        elif row['2'] == player:
          ids.append(row['PlayerId'])
        elif row['3'] == player:
          ids.append(row['PlayerId'])
        else:
          continue
    df['pids'] = ids
    return df

In [ ]:
# This function scrapes weekly fantasy football scores by player
def weeklydata(year,start,end,pids):
  import time
  import pandas as pd
  from bs4 import BeautifulSoup
  import requests
  #Scrape and create dataframe from website
  url = f'https://www.fantasypros.com/nfl/reports/leaders/half-ppr.php?year={year}&start={start}&end={end}'
  df = pd.read_html(url,header=0)[0]
  df = df[df['Pos'] != "DST"]
  df = df.fillna(0)
  ids = []
  for index, row in df.iterrows():
    player = row['Player']
    for index, row in pids.iterrows():
      if row['1'] == player:
        ids.append(row['PlayerId'])
      elif row['2'] == player:
        ids.append(row['PlayerId'])
      elif row['3'] == player:
        ids.append(row['PlayerId'])
      else:
        continue
  df['pids'] = ids
  df = df.drop(['#'],axis=1)
  columns = ['pids','Player','Pos','Team','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','AVG','TTL']
  df = df[columns]
  numcol = df.columns[4:]
  for col in numcol:
    df[col] = pd.to_numeric(df[col],errors='coerce')
  df = df.fillna(0)
  return df

In [ ]:
# This function scrapes interceptions in order to validate the true .5 PPR score considering the weekly scores from fantasy pros doesn't score interceptions the same as ESPN
def intscrape(week,pids):
  import time
  import pandas as pd
  from bs4 import BeautifulSoup
  import requests
  from functools import reduce
  df_list = []
  for num in range(1,week+1):
    name = f'df_{num}'
    url = f'https://www.fantasypros.com/nfl/stats/qb.php?week={num}&range=week'
    df = pd.read_html(url,header=1)[0]
    df = df[df['ATT'] > 0]
    df = df.replace('\(.*\)','',regex=True).astype(object)
    # Strips extra whitespace from end of player column
    df['Player'] = df['Player'].str.rstrip()
    # Update total tds if null
    df = df[['Player','INT']]
    ids = []
    pidname = []
    for index, row in df.iterrows():
      player = row['Player']
      for index, row in pids.iterrows():
        if row['1'] == player:
          ids.append(row['PlayerId'])
          pidname.append(row['1'])
        elif row['2'] == player:
          ids.append(row['PlayerId'])
          pidname.append(row['1'])
        elif row['3'] == player:
          ids.append(row['PlayerId'])
          pidname.append(row['1'])
        else:
          continue
    df['pids'] = ids
    df['Name'] = pidname
    df = df[['pids','Name','INT']]
    colname1 = 'Week_'+str(num)
    #colname2 = 'Player_'+str(num)
    df.rename(columns={'INT': colname1},inplace=True)
    globals()[name] = df
    df_list.append(globals()[name])
  int_df = reduce(lambda x, y: pd.merge(x, y, how='outer', on = ['pids','Name']), df_list)
  int_df = int_df.fillna(0)
  numcol = int_df.columns[2:]
  for col in numcol:
    int_df[col] = pd.to_numeric(int_df[col],errors='coerce')
  return int_df

In [ ]:
# This used the ESPN API to scrape our league data. I've made the code generic below
def ffballleague(week,pids):
  !pip install espn.api
  import pandas as pd
  from espn_api.football import League
  league = League(league_id='LEAGUEID',year='YEAR',
                espn_s2='INSERTHERE',
                swid='{INSERTHERE}')
  team1 = str(league.teams[0])
  team2 = str(league.teams[1])
  team3 = str(league.teams[2])
  team4 = str(league.teams[3])
  team5 = str(league.teams[4])
  team6 = str(league.teams[5])
  team7 = str(league.teams[6])
  team8 = str(league.teams[7])
  team9 = str(league.teams[8])
  team10 = str(league.teams[9])
  gm_dict = {'GM1':team1,
           'GM2':team2,
           'GM3':team3,
           'GM4':team4,
           'GM5':team5,
           'GM6':team6,
           'GM7':team7,
           'GM8':team8,
           'GM9':team9,
           'GM10':team10}
  leaguedf = pd.DataFrame()
  weeknum = []
  team = []
  gms = []
  place = []
  player = []
  pos = []
  tm = []
  playerproj = []
  playerscore = []
  opp = []
  for num in range(1,week+1):
    week = num
    weekinfo = league.box_scores(num)
    for num1 in range(0,5):
      hometeam = str(weekinfo[num1].home_team)
      teamsize = len(weekinfo[num1].home_lineup)
      for key, value in gm_dict.items():
        if hometeam == value:
          gm = key
      for num2 in range(0,teamsize):
          homeplayer = str(weekinfo[num1].home_lineup[num2].name)
          homepos = str(weekinfo[num1].home_lineup[num2].slot_position)
          if homepos in ('BE','IR'):
            continue
          else:
            place.append('Home')
            weeknum.append(week)
            team.append(hometeam)
            gms.append(gm)
            player.append(str(weekinfo[num1].home_lineup[num2].name))
            playerproj.append(weekinfo[num1].home_lineup[num2].projected_points)
            playerscore.append(weekinfo[num1].home_lineup[num2].points)
            pos.append(str(weekinfo[num1].home_lineup[num2].slot_position))
            tm.append(str(weekinfo[num1].home_lineup[num2].proTeam))
            opp.append(str(weekinfo[num1].away_team))
    for num1 in range(0,5):
      awayteam = str(weekinfo[num1].away_team)
      teamsize = len(weekinfo[num1].away_lineup)
      for key, value in gm_dict.items():
        if awayteam == value:
          gm = key
      for num2 in range(0,teamsize):
          awayplayer = str(weekinfo[num1].away_lineup[num2].name)
          awaypos = str(weekinfo[num1].away_lineup[num2].slot_position)
          if awaypos in ('BE','IR'):
            continue
          else:
            place.append('Away')
            weeknum.append(week)
            team.append(awayteam)
            gms.append(gm)
            player.append(str(weekinfo[num1].away_lineup[num2].name))
            playerproj.append(weekinfo[num1].away_lineup[num2].projected_points)
            playerscore.append(weekinfo[num1].away_lineup[num2].points)
            pos.append(str(weekinfo[num1].away_lineup[num2].slot_position))
            tm.append(str(weekinfo[num1].away_lineup[num2].proTeam))
            opp.append(str(weekinfo[num1].home_team))
  leaguedf['week'] = weeknum
  leaguedf['team'] = team
  leaguedf['gm'] = gms
  leaguedf['home/away'] = place
  leaguedf['player'] = player
  leaguedf['position'] = pos
  leaguedf['proteam'] = tm
  leaguedf['playerproj'] = playerproj
  leaguedf['playerscore'] = playerscore
  leaguedf['difference'] = leaguedf['playerscore'] - leaguedf['playerproj']
  leaguedf['opponent'] = opp
  leaguedf['position'] = leaguedf['position'].replace({'RB/WR/TE':'FLEX'})
  leaguedf['team'] = leaguedf['team'].str[5:]
  leaguedf['team'] = leaguedf['team'].str[:-1]
  leaguedf['team'] = leaguedf['team'].str.rstrip()
  leaguedf['team'] = leaguedf['team'].str.lstrip()
  leaguedf['opponent'] = leaguedf['opponent'].str[5:]
  leaguedf['opponent'] = leaguedf['opponent'].str[:-1]
  leaguedf['opponent'] = leaguedf['opponent'].str.rstrip()
  leaguedf['opponent'] = leaguedf['opponent'].str.lstrip()
  leaguedf['proteam'] = leaguedf['proteam'].replace({'JAX':'JAC','WSH':'WAS','OAK':'LV'})
  ids = []
  for index, row in leaguedf.iterrows():
    player = row['player']
    for index, row in pids.iterrows():
      if row['1'] == player:
        ids.append(row['PlayerId'])
      elif row['2'] == player:
        ids.append(row['PlayerId'])
      elif row['3'] == player:
        ids.append(row['PlayerId'])
      else:
        continue
  leaguedf['pids'] = ids
  leaguedf = leaguedf[['week','team','gm','home/away','player','pids','position','proteam','playerproj','playerscore','difference','opponent']]
  return leaguedf

In [ ]:
# This function scarpes the fantasy pros website and obtains the staistics for each player to calculate number of opportunities on a weekly basis (Pass Atts, Rushes, Targets)
def oppscrape(week,pids):
    import time
    import pandas as pd
    from bs4 import BeautifulSoup
    import requests
    from functools import reduce
    df_list = []
    for num in range(1,week+1):
        print(f'Week {num}')
        week_num = num
        name = f'df_{num}'

        url = f'https://www.fantasypros.com/nfl/stats/qb.php?week={num}&range=week'
        df0 = pd.read_html(url,header=1)[0]
        df0 = df0[(df0['ATT'] > 0) | (df0['ATT.1'] > 0)]
        df0 = df0.replace('\(.*\)','',regex=True).astype(object)
        # Strips extra whitespace from end of player column
        df0['Player'] = df0['Player'].str.rstrip()
        # Update total tds if null
        df0 = df0[['Player','ATT','TD','ATT.1','TD.1']]
        ids = []
        pidname = []
        for index, row in df0.iterrows():
            player = row['Player']
            for index, row in pids.iterrows():
                if row['1'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['2'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['3'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                else:
                    continue
        df0['pids'] = ids
        df0['Name'] = pidname
        df0['Pos'] = 'QB'
        df0['TGT'] = 0
        df0['RecTD'] = 0
        df0['DEF TD'] = 0
        df0['Week'] = week_num
        df0.rename(columns={'ATT': 'PassAtt','TD':'PassTD','ATT.1':'RushAtt','TD.1':'RushTD'},inplace=True)
        df0 = df0[['Week','pids','Name','Pos','PassAtt','PassTD','RushAtt','RushTD','TGT','RecTD','DEF TD']]

        url = f'https://www.fantasypros.com/nfl/stats/rb.php?week={num}&range=week'
        df1 = pd.read_html(url,header=1)[0]
        df1 = df1[(df1['ATT'] > 0) | (df1['TGT'] > 0)]
        df1 = df1.replace('\(.*\)','',regex=True).astype(object)
        # Strips extra whitespace from end of player column
        df1['Player'] = df1['Player'].str.rstrip()
        df1 = df1[['Player','ATT','TD','TGT','TD.1']]
        ids = []
        pidname = []
        for index, row in df1.iterrows():
            player = row['Player']
            for index, row in pids.iterrows():
                if row['1'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['2'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['3'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                else:
                    continue
        df1['pids'] = ids
        df1['Name'] = pidname
        df1['Pos'] = 'RB'
        df1['PassAtt'] = 0
        df1['PassTD'] = 0
        df1['DEF TD'] = 0
        df1['Week'] = week_num
        df1.rename(columns={'ATT': 'RushAtt','TD':'RushTD','TD.1':'RecTD'},inplace=True)
        df1 = df1[['Week','pids','Name','Pos','PassAtt','PassTD','RushAtt','RushTD','TGT','RecTD','DEF TD']]

        url = f'https://www.fantasypros.com/nfl/stats/wr.php?week={num}&range=week'
        df2 = pd.read_html(url,header=1)[0]
        df2 = df2[(df2['TGT'] > 0) | (df2['ATT'] > 0)]
        df2 = df2.replace('\(.*\)','',regex=True).astype(object)
        # Strips extra whitespace from end of player column
        df2['Player'] = df2['Player'].str.rstrip()
        df2 = df2[['Player','TGT','TD','ATT','TD.1']]
        ids = []
        pidname = []
        for index, row in df2.iterrows():
            player = row['Player']
            for index, row in pids.iterrows():
                if row['1'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['2'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['3'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                else:
                    continue
        df2['pids'] = ids
        df2['Name'] = pidname
        df2['Pos'] = 'WR'
        df2['PassAtt'] = 0
        df2['PassTD'] = 0
        df2['DEF TD'] = 0
        df2['Week'] = week_num
        df2.rename(columns={'TD':'RecTD','ATT': 'RushAtt','TD.1':'RushTD'},inplace=True)
        df2 = df2[['Week','pids','Name','Pos','PassAtt','PassTD','RushAtt','RushTD','TGT','RecTD','DEF TD']]

        url = f'https://www.fantasypros.com/nfl/stats/te.php?week={num}&range=week'
        df3 = pd.read_html(url,header=1)[0]
        df3 = df3[(df3['TGT'] > 0) | (df3['ATT'] > 0)]
        df3 = df3.replace('\(.*\)','',regex=True).astype(object)
        # Strips extra whitespace from end of player column
        df3['Player'] = df3['Player'].str.rstrip()
        df3 = df3[['Player','TGT','TD','ATT','TD.1']]
        ids = []
        pidname = []
        for index, row in df3.iterrows():
            player = row['Player']
            for index, row in pids.iterrows():
                if row['1'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['2'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['3'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                else:
                    continue
        df3['pids'] = ids
        df3['Name'] = pidname
        df3['Pos'] = 'TE'
        df3['PassAtt'] = 0
        df3['PassTD'] = 0
        df3['DEF TD'] = 0
        df3['Week'] = week_num
        df3.rename(columns={'TD':'RecTD','ATT': 'RushAtt','TD.1':'RushTD'},inplace=True)
        df3 = df3[['Week','pids','Name','Pos','PassAtt','PassTD','RushAtt','RushTD','TGT','RecTD','DEF TD']]

        url = f'https://www.fantasypros.com/nfl/stats/dst.php?week={num}&range=week'
        df4 = pd.read_html(url,header=0)[0]
        df4 = df4.replace('\(.*\)','',regex=True).astype(object)
        # Strips extra whitespace from end of player column
        df4['Player'] = df4['Player'].str.rstrip()
        df4 = df4[['Player','DEF TD','SPC TD']]
        df4.rename(columns={'DEF TD':'deftd'},inplace=True)
        ids = []
        pidname = []
        for index, row in df4.iterrows():
            player = row['Player']
            for index, row in pids.iterrows():
                if row['1'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['2'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                elif row['3'] == player:
                    ids.append(row['PlayerId'])
                    pidname.append(row['1'])
                else:
                    continue
        df4['pids'] = ids
        df4['Name'] = pidname
        df4['Pos'] = 'DST'
        df4['PassAtt'] = 0
        df4['PassTD'] = 0
        df4['RushAtt'] = 0
        df4['RushTD'] = 0
        df4['TGT'] = 0
        df4['RecTD'] = 0
        df4['Week'] = week_num
        df4['DEF TD'] = df4['deftd'] + df4['SPC TD']
        df4 = df4[['Week','pids','Name','Pos','PassAtt','PassTD','RushAtt','RushTD','TGT','RecTD','DEF TD']]

        df = pd.concat([df0,df1,df2,df3,df4]).reset_index(drop=True)

        globals()[name] = df
        df_list.append(globals()[name])

    opp_df = pd.concat(df_list).reset_index(drop=True)
    #opp_df = reduce(lambda x, y: pd.merge(x, y, how='outer', on = ['pids','Name']), df_list)
    #opp_df = opp_df.fillna(0)
    #numcol = opp_df.columns[2:]
    #for col in numcol:
        #int_df[col] = pd.to_numeric(int_df[col],errors='coerce')
    return opp_df

In [ ]:
# This function runs a montecarlo simulation 10,000 times to calculate playoff probabilities given mean score and standard deviation
def league_sim():
  import time
  import numpy as np
  import pandas as pd
  !pip install espn.api
  from espn_api.football import League
  league = League(league_id='LEAGUEID',year='YEAR',
                espn_s2='INSERTHERE',
                swid='{INSERTHERE}')
  agg_tm = []
  agg_wins = []
  agg_loss = []
  agg_top = []
  agg_dps = []
  if_playoffs = []
  for num in range(10000):
      if num % 1000 == 0:
          print(num)
      team_name = []
      tm_score = []
      opponent = []
      oppon_score = []
      wk = []
      for week in range(1,15):
          for team in range(1,11):
              tm = league.get_team_data(team)
              score = tm.scores[week-1]
              opp = tm.schedule[week-1]
              opp_score = opp.scores[week-1]
              tm1 = str(tm)
              opp1 = str(opp)
              wk.append(week)
              team_name.append(tm1)
              tm_score.append(score)
              opponent.append(opp1)
              oppon_score.append(opp_score)


      df = pd.DataFrame({'Week':wk,
                        'Team':team_name,
                        'Score':tm_score,
                        'Opponent':opponent,
                        'Opp Score':oppon_score})
      df['Team'] = df['Team'].str[5:]
      df['Team'] = df['Team'].str[:-1]
      df['Opponent'] = df['Opponent'].str[5:]
      df['Opponent'] = df['Opponent'].str[:-1]
      tms = df.query("Score > 0").groupby('Team').agg({'Score':['mean','std']}).reset_index()
      wks_remaining = df.query("Score == 0").groupby('Week')['Week'].nunique().sum()
      cur_week = df[df['Score'] == 0].reset_index()['Week'][0]
      newscores = []
      for wk in range(cur_week,cur_week+wks_remaining):
          tms_work = tms.copy()
          prediction = []
          for index, row in tms_work.iterrows():
              newscore = np.random.normal(row['Score']['mean'],row['Score']['std'])
              prediction.append(newscore)
          tms_work['Pred'] = prediction
          for index, row in df.iterrows():
              if row['Week'] == wk:
                  for index1, row1 in tms_work.iterrows():
                      if row1['Team'][0] == row['Team']:
                          df._set_value(index,'Score',row1['Pred'][0])
                      else:
                          continue
                  for index1, row1 in tms_work.iterrows():
                      if row1['Team'][0] == row['Opponent']:
                          df._set_value(index,'Opp Score',row1['Pred'][0])
              else:
                  continue
      medians = df.query("Score > 0").groupby('Week')['Score'].median().reset_index()
      win_check = []
      for index, row in df.iterrows():
          if row['Score'] > row['Opp Score']:
              win_check.append(1)
          else:
              win_check.append(0)
      df['Win/Loss'] = win_check
      top = []
      for index, row in df.iterrows():
          if row['Score'] > 0:
              week = row['Week']
              if row['Score'] >= medians[medians['Week'] == week]['Score'][week-1]:
                  top.append(1)
              else:
                  top.append(0)
          else:
              top.append(0)
      df['Top 5'] = top
      standings = df.groupby('Team')[['Win/Loss','Top 5']].sum().reset_index()
      standings['DPS'] = standings['Win/Loss'] + standings['Top 5']
      standings['W'] = standings['Win/Loss']
      standings['L'] = 14 - standings['W']
      standings = standings[['Team','W','L','Top 5','DPS']]
      standings = standings.sort_values('DPS',ascending=False).reset_index(drop=True)
      playoffs = []
      for index, row in standings.iterrows():
          if index <= 5:
              playoffs.append(1)
          else:
              playoffs.append(0)
      standings['Playoffs'] = playoffs
      for index, row in standings.iterrows():
          agg_tm.append(row['Team'])
          if_playoffs.append(row['Playoffs'])
          agg_wins.append(row['W'])
          agg_loss.append(row['L'])
          agg_top.append(row['Top 5'])
          agg_dps.append(row['DPS'])

  mass_data = pd.DataFrame({'Team':agg_tm,
                            'W': agg_wins,
                            'L': agg_loss,
                            'Top 5': agg_top,
                            'DPS': agg_dps,
                            'Playoffs':if_playoffs})
  projections = mass_data.groupby('Team').aggregate({'W':'mean','L':'mean','Top 5':'mean','DPS':'mean','Playoffs':'sum'})
  projections = projections.reset_index()
  projections = projections.sort_values('Playoffs',ascending=False).reset_index(drop=True)
  projections['Playoffs'] = projections['Playoffs']/100
  return projections

### Run Code

In [ ]:
# let colab access my google drive
# unique player ids
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
pids = pd.read_csv('/content/drive/MyDrive/FILELOCATION')
pids = pids.fillna("")
pids

Mounted at /content/drive


,PlayerId,1,2,3
0,ABro00,A.J. Brown,AJ Brown,
1,AGre00,A.J. Green,AJ Green,
2,APer00,A.T. Perry,AT Perry,
3,AJon00,Aaron Jones,,
4,ARod00,Aaron Rodgers,,
...,...,...,...,...
927,Seahawks,Seahawks,Seahawks D/ST,Seattle Seahawks
928,49ers,49ers,49ers D/ST,San Francisco 49ers
929,Buccaneers,Buccaneers,Buccaneers D/ST,Tampa Bay Buccaneers
930,Titans,Titans,Titans D/ST,Tennessee Titans


In [ ]:
import pandas as pd
df = prodata(2023,pids)
df1 = weeklydata(2023,1,18,pids)
df2 = intscrape(15,pids)
df3 = ffballleague(15,pids)
df4 = oppscrape(15,pids)
#df5 = league_sim()

Estimated time to completion: 2.8 minutes
Joshua Dobbs


<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)
<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)


Cam Akers


<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)
<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)
<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)


Tony Jones


...

Deon Jackson


<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)
<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)
<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)


Gunner Olszewski


<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)
<ipython-input-1-5105258f5983>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ddf = ddf.append(pd.Series(row_value, index=ddf.columns[:len(row_value)]), ignore_index=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 821.1 kB/s eta 0:00:00
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
Week 9
Week 10
Week 11
Week 12
Week 13
Week 14
Week 15


In [ ]:
# Input the data directly into a google sheet. Do different iterations by dataframe and what sheet they need to be input into
from google.colab import auth
from google.auth import default

try:
  import gspread
except ModuleNotFoundError:
  if 'google.colab' in str(get_ipython()):
    %pip install gspread
  import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open("FILENAME") #Update file name
sheet1 = worksheet.worksheet("SHEETNAME") #Update the sheet name the df will go to

sheet1.update([df.columns.values.tolist()] + df.fillna(-1).values.tolist()) #Update the df to which df you want to send to the sheet (df, df1, df2, etc)

{'spreadsheetId': '1dfnUdodzFDeBrFG7EdVaL6zgPM1PclW6aYgxiO1J-7A',
 'updatedRange': 'ProData!A1:BB564',
 'updatedRows': 564,
 'updatedColumns': 54,
 'updatedCells': 30456}